<a href="https://colab.research.google.com/github/cryoTUD/ColabScale/blob/main/ColabScale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://gitlab.tudelft.nl/aj-lab/locscale/-/raw/master/doc/img/LocScale_logo.png" height="200" align="right" style="height:240px">

#```ColabScale```

Easy to use cryo-EM map sharpening using [```LocScale```](https://gitlab.tudelft.nl/aj-lab/locscale) and generation of feature-enhanced maps with [```LocScale-EMmerNet```](https://gitlab.tudelft.nl/aj-lab/emmernet).

For more details, see <a href="#Instructions">bottom</a> of the notebook and read our manuscripts.


In [1]:
# @title Setup environment
%%capture
%%time
import os

if not os.path.isfile("CONDA_READY"):
   print("installing conda...")
   os.system("pip install -q condacolab")
   import condacolab
   condacolab.install()
   os.system("conda install -c conda-forge gfortran")
   os.system("conda create -n locscale python=3.8")
   os.system("touch CONDA_READY")

# downgrade CUDA version to 11.1 (for compatibility with tf model)
!wget https://developer.download.nvidia.com/compute/cuda/11.1.0/local_installers/cuda-repo-ubuntu2004-11-1-local_11.1.0-455.23.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu2004-11-1-local_11.1.0-455.23.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu2004-11-1-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-11-1
!export CUDA_PATH=/usr/local/cuda-11.1/
!nvcc --version

In [1]:
# @title Install LocScale2
%%time
%%capture
%%bash
# install CCP4 monomer lib
if [ -d "/usr/local/monomer_lib" ]; then
  echo "removing existing monomer library"
  rm -rf /usr/local/monomer_lib
fi
mkdir /usr/local/monomer_lib
git clone https://github.com/MonomerLibrary/monomers.git /usr/local/monomer_lib
export CLIBD_MON=/usr/local/monomer_lib/
# install locscale
eval "$(conda shell.bash hook)" # copy conda command to shell
conda activate locscale
pip install ipykernel
python --version
pip install -q --no-warn-conflicts locscale


CPU times: user 520 ms, sys: 105 ms, total: 625 ms
Wall time: 2min 26s


In [ ]:
#@markdown ### Input

from google.colab import files
import os

jobname = 'test' #@param {type:"string"}
job_type = "feature_enhance" # @param ["model-based", "model-free", "hybrid", "feature_enhance"]
#@markdown - __model-based__: ```LocScale``` sharpening using atomic model
#@markdown - __model-free__: ```LocScale``` sharpening without atomic model
#@markdown - __hybrid__: ```LocScale``` sharpening with partial atomic model
#@markdown - __feature_enhance__: Confidence-aware density modification with ```LocScale-EMmerNet```


use_model = True #@param {type:"boolean"}
if use_model == True:
  input_model_path = os.path.join(jobname,f"model")
  os.makedirs(input_model_path, exist_ok=True)
  print("Please select model file...")
  uploaded = files.upload()
  for model in uploaded.keys():
     if model.endswith('.pdb'):
        os.rename(model,os.path.join(input_model_path,model))
     else:
        print("Uploaded file is not a PDB file; please select correct file...")
        os.remove(model)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_model_path,model))
     use_model = True
elif use_model == False:
   input_model_path = None

input_model_path = "/content/map.mrc"

use_half_maps = True #@param {type:"boolean"}
if use_half_maps == True:
  input_half_map_path = os.path.join(jobname,f"half_maps")
  os.makedirs(input_half_map_path, exist_ok=True)
  print("Please select half maps...")
  uploaded = files.upload()
  for map in uploaded.keys():
     if map.endswith('.mrc'):
        os.rename(map,os.path.join(input_half_map_path,map))
     else:
        print("Uploaded file is not a MRC file; please select correct file...")
        os.remove(map)
        uploaded = files.upload()
        os.rename(model,os.path.join(input_half_map_path,map))
     use_half_maps = True
elif use_half_maps == False:
   input_half_map_path = None


In [ ]:
#@markdown ## Advanced Options

from google.colab import files
import os

#@markdown Most of these options should be left at default. Please only change if necessary and if you know what you are doing.

#@markdown #### FDR options

fdr_threshold = 0.01 #@param {type:"string"}
fdr_window_size = None #@param {type:"string"}
fdr_filter = None #@param {type:"string"}
averaging_filter_size = 3 #@param {type:"string"}
mask_threshold = 0.99 #@param {type:"string"}

#@markdown \

#@markdown #### EMmerNet options
model_path = None #@param {type:"string"}
low_context_model = False #@param {type:"boolean"}
batch_size = 8 #@param {type:"string"}
cube_size = 32 #@param {type:"string"}
gpu_ids = None #@param {type:"string"}

#@markdown \

#@markdown #### Reference options

model_resolution = None #@param {type:"string"}
symmetry = "C1" #@param {type:"string"}





In [ ]:
#@markdown ## Some useful stuff for testing
#!pip install gpustat
#!gpustat

#INFO of CPUs
from psutil import *
print("Number of CPU: ", cpu_count())
#!cat /proc/cpuinfo

#INFO ON GPUs
import tensorflow as tf
tf.config.list_physical_devices('GPU')

#from tensorflow.python.client import device_lib
#tf.test.is_gpu_available()

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
print(get_available_gpus())

#! nvidia-smi
#!nvcc --version


Number of CPU:  12
['/device:GPU:0']


In [ ]:
%%time
%%bash
eval "$(conda shell.bash hook)"
conda activate locscale
#pip install ipykernel
/usr/local/envs/locscale/bin/locscale feature_enhance -np 8 -em map.mrc -ma mask.mrc -v -o test_fe_sym.mrc -gpus 0 -sym C4
#python
#from locscale.automate.tools import get_defaults_dictionary
#print(get_defaults_dictionary())